# Défi Grosses Data 2018
# Algorithme XGBoost

## Préparation des données

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from os import listdir
import Annex

Importe les données concaténées:

In [15]:
df=Annex.get_data_raw()
df.shape

(189280, 31)

Importe les données séparées par type:

In [16]:
meteo_quant, meteo_qual, meteo_date, meteo_y=Annex.get_data_tidied()

Remplace les variables qualitatives par leur indicatrices. 

**Attention, ici seuls "mois" et "insee" sont considérés qualitatives!** (what about the wind? what about the "ech"?)

In [17]:
df_dummies=pd.get_dummies(df[['mois','insee']])
df_full_qtt=pd.concat([df,df_dummies],axis=1)
df_full_qtt=df_full_qtt.drop(['mois','insee'],axis=1)

Sépare les échantillons d'apprentissage et de test

In [113]:
from sklearn.model_selection import train_test_split
Y=df_full_qtt['tH2_obs']
X=df_full_qtt
X=X.drop(['tH2_obs','date'],axis=1) ## !!! Date?
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=11)
X_train.shape, X_test.shape

((132496, 46), (56784, 46))

## XGBoost

### Adaptation du format des données pour XGBoost

In [114]:
import xgboost as xgb

We first need to transform panda dataframe into numpy 2D-array :

In [115]:
X_train=pd.DataFrame.as_matrix(X_train)
X_test=pd.DataFrame.as_matrix(X_test)
Y_train=pd.DataFrame.as_matrix(Y_train)
Y_test=pd.DataFrame.as_matrix(Y_test)

In [117]:
X_train
Y_test

array([ -1.39999998,   6.5999999 ,   8.89999962, ...,  15.        ,
        23.70000076,   7.0999999 ])

The data need to be stored into a DMatrix (cf xgboost documentation) : 

In [121]:
X_train = xgb.DMatrix(X_train, label=Y_train)
X_test = xgb.DMatrix(X_test, label=Y_test)

In [122]:
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'reg:linear'}
param['eval_metric'] ='auc'

In [123]:
num_round = 10
bst = xgb.train(param,X_train,num_round)

Make prediction

In [124]:
score = bst.eval(X_test)
Y_pred = bst.predict(X_test)

In [125]:
#from sklearn.metrics import mean_squared_error
#mean_squared_error(Y_test, Y_pred)
Y_pred

array([ nan,  nan,  nan, ...,  nan,  nan,  nan], dtype=float32)